In [45]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 20: A Regular Map ---</h2><p>While you were learning about instruction pointers, the Elves made considerable progress. When you look up, you discover that the North Pole base construction project has completely surrounded you.</p>
<p>The area you are in is made up entirely of <em>rooms</em> and <em>doors</em>. The rooms are arranged in a grid, and rooms only connect to adjacent rooms when a door is present between them.</p>
<p>For example, drawing rooms as <code>.</code>, walls as <code>#</code>, doors as <code>|</code> or <code>-</code>, your current position as <code>X</code>, and where north is up, the area you're in might look like this:</p>
<pre><code>#####
#.|.#
#-###
#.|X#
#####
</code></pre>
<p>You get the attention of a passing construction Elf and ask for a map. "I don't have time to draw out a map of this place - it's <em>huge</em>. Instead, I can give you directions to <em>every room in the facility</em>!" He writes down some directions on a piece of parchment and runs off. In the example above, the instructions might have been <code>^WNE$</code>, a <a href="https://en.wikipedia.org/wiki/Regular_expression">regular expression</a> or "<em>regex</em>" (your puzzle input).</p>
<p>The regex matches routes (like <code>WNE</code> for "west, north, east") that will take you from your current room through various doors in the facility. In aggregate, the routes will take you through <em>every door in the facility at least once</em>; mapping out all of these routes will let you build a proper map and find your way around.</p>
<p><code>^</code> and <code>$</code> are at the beginning and end of your regex; these just mean that the regex doesn't match anything outside the routes it describes. (Specifically, <code>^</code> matches the start of the route, and <code>$</code> matches the end of it.) These characters will not appear elsewhere in the regex.</p>
<p>The rest of the regex matches various sequences of the characters <code>N</code> (north), <code>S</code> (south), <code>E</code> (east), and <code>W</code> (west). In the example above, <code>^WNE$</code> matches only one route, <code>WNE</code>, which means you can move <em>west, then north, then east</em> from your current position. Sequences of letters like this always match that exact route in the same order.</p>
<p>Sometimes, the route can <em>branch</em>. A branch is given by a <em>list of options</em> separated by pipes (<code>|</code>) and wrapped in parentheses. So, <code>^N(E|W)N$</code> contains a branch: after going north, you must choose to go <em>either east or west</em> before finishing your route by going north again. By tracing out the possible routes after branching, you can determine where the doors are and, therefore, where the rooms are in the facility.</p>
<p>For example, consider this regex: <code>^ENWWW(NEEE|SSE(EE|N))$</code></p>
<p>This regex begins with <code>ENWWW</code>, which means that from your current position, all routes must begin by moving east, north, and then west three times, in that order. After this, there is a branch.  Before you consider the branch, this is what you know about the map so far, with doors you aren't sure about marked with a <code>?</code>:</p>
<pre><code>#?#?#?#?#
?.|.|.|.?
#?#?#?#-#
    ?X|.?
    #?#?#
</code></pre>
<p>After this point, there is <code>(NEEE|SSE(EE|N))</code>. This gives you exactly two options: <code>NEEE</code> and <code>SSE(EE|N)</code>. By following <code>NEEE</code>, the map now looks like this:</p>
<pre><code>#?#?#?#?#
?.|.|.|.?
#-#?#?#?#
?.|.|.|.?
#?#?#?#-#
    ?X|.?
    #?#?#
</code></pre>
<p>Now, only <code>SSE(EE|N)</code> remains. Because it is in the same parenthesized group as <code>NEEE</code>, it starts from the same room <code>NEEE</code> started in. It states that starting from that point, there exist doors which will allow you to move south twice, then east; this ends up at another branch. After that, you can either move east twice or north once. This information fills in the rest of the doors:</p>
<pre><code>#?#?#?#?#
?.|.|.|.?
#-#?#?#?#
?.|.|.|.?
#-#?#?#-#
?.?.?X|.?
#-#-#?#?#
?.|.|.|.?
#?#?#?#?#
</code></pre>
<p>Once you've followed all possible routes, you know the remaining unknown parts are all walls, producing a finished map of the facility:</p>
<pre><code>#########
#.|.|.|.#
#-#######
#.|.|.|.#
#-#####-#
#.#.#X|.#
#-#-#####
#.|.|.|.#
#########
</code></pre>
<p>Sometimes, a list of options can have an <em>empty option</em>, like <code>(NEWS|WNSE|)</code>. This means that routes at this point could effectively skip the options in parentheses and move on immediately.  For example, consider this regex and the corresponding map:</p>
<pre><code>^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$

###########
#.|.#.|.#.#
#-###-#-#-#
#.|.|.#.#.#
#-#####-#-#
#.#.#X|.#.#
#-#-#####-#
#.#.|.|.|.#
#-###-###-#
#.|.|.#.|.#
###########
</code></pre>

<p>This regex has one main route which, at three locations, can optionally include additional detours and be valid: <code>(NEWS|)</code>, <code>(WNSE|)</code>, and <code>(SWEN|)</code>. Regardless of which option is taken, the route continues from the position it is left at after taking those steps. So, for example, this regex matches all of the following routes (and more that aren't listed here):</p>
<ul>
<li><code>ENNWSWWSSSEENEENNN</code></li>
<li><code>ENNWSWW<em>NEWS</em>SSSEENEENNN</code></li>
<li><code>ENNWSWW<em>NEWS</em>SSSEENEE<em>SWEN</em>NNN</code></li>
<li><code>ENNWSWWSSSEEN<em>WNSE</em>EENNN</code></li>
</ul>
<p>By following the various routes the regex matches, a full map of all of the doors and rooms in the facility can be assembled.</p>
<p>To get a sense for the size of this facility, you'd like to determine which room is <em>furthest</em> from you: specifically, you would like to find the room for which the <em>shortest path to that room would require passing through the most doors</em>.</p>
<ul>
<li>In the first example (<code>^WNE$</code>), this would be the north-east corner <code><em>3</em></code> doors away.</li>
<li>In the second example (<code>^ENWWW(NEEE|SSE(EE|N))$</code>), this would be the south-east corner <code><em>10</em></code> doors away.</li>
<li>In the third example (<code>^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$</code>), this would be the north-east corner <code><em>18</em></code> doors away.</li>
</ul>
<p>Here are a few more examples:</p>
<pre><code>Regex: ^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$
Furthest room requires passing 23 doors

#############
#.|.|.|.|.|.#
#-#####-###-#
#.#.|.#.#.#.#
#-#-###-#-#-#
#.#.#.|.#.|.#
#-#-#-#####-#
#.#.#.#X|.#.#
#-#-#-###-#-#
#.|.#.|.#.#.#
###-#-###-#-#
#.|.#.|.|.#.#
#############
</code></pre>

<pre><code>Regex: ^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$
Furthest room requires passing 31 doors

###############
#.|.|.|.#.|.|.#
#-###-###-#-#-#
#.|.#.|.|.#.#.#
#-#########-#-#
#.#.|.|.|.|.#.#
#-#-#########-#
#.#.#.|X#.|.#.#
###-#-###-#-#-#
#.|.#.#.|.#.|.#
#-###-#####-###
#.|.#.|.|.#.#.#
#-#-#####-#-#-#
#.#.|.|.|.#.|.#
###############
</code></pre>
<p><em>What is the largest number of doors you would be required to pass through to reach a room?</em> That is, find the room for which the shortest path from your starting location to that room would require passing through the most doors; what is the fewest doors you can pass through to reach it?</p>
</article>


In [46]:
from collections import deque

from more_itertools import minmax


tests = [
    {
        "name": "Example 1",
        "regex": "^$",
        "expected": 0,
    },
    {
        "name": "Example 2",
        "regex": "^W$",
        "expected": 1,
    },
    {
        "name": "Example 3",
        "regex": "^WNE$",
        "expected": 3,
    },
    {
        "name": "Example 4",
        "regex": "^N(E|W)N$",
        #
        #    E
        #   / \
        # ^N   N$
        #   \ /
        #    W
        #
        "expected": 3,
    },
    {
        "name": "Example 5",
        "regex": "^N(EE|W|N)N$",
        "expected": 4,
    },
    {
        "name": "Example 6",
        "regex": "^N(EE|W|)N$",
        "expected": 2,
    },
    {
        "name": "Example 7",
        "regex": "^ENWWW(NEEE|SSE(EE|N))$",
        #
        #        NEEE------\
        #       /           \
        # ^ENWWW     SSE     $
        #       \   /   \   /
        #        SSE     --/
        #           \   /
        #            N--
        #
        "expected": 10,
    },
    {
        "name": "Example 8",
        "regex": "^ENWWW((EE|N)NEEE|SSE)$",
        #
        #          SSE
        #         /   \
        #        /     NEEE
        #       / \   /    \
        # ^ENWWW   N-       $
        #       \          /
        #        SSE------/
        #
        "expected": 11,
    },
    {
        "name": "Example 9",
        "regex": "^(NEW|WNSE|)$",
        "expected": 0,
    },
    {
        "name": "Example 10",
        "regex": "^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$",
        "expected": 18,
    },
    {
        "name": "Example 11",
        "regex": "^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$",
        "expected": 23,
    },
    {
        "name": "Example 12",
        "regex": "^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$",
        "expected": 31,
    },
]


def shortest_path_with_most_doors(regex: str) -> int:
    i = 0
    lengths = deque([deque([0])])

    while regex[i] != "$":
        if regex[i] in "NESW":
            lengths[-1][-1] += 1
        elif regex[i] == "(":
            lengths.append(deque([0]))
        elif regex[i] == "|":
            lengths[-1].append(0)
        elif regex[i] == ")":
            ls = lengths.pop()
            min_ls, max_ls = minmax(ls)
            if min_ls > 0:
                lengths[-1][-1] += max_ls

        i += 1

    return lengths[0][-1]


run_tests_params(shortest_path_with_most_doors, tests)


Test Example 1 passed, for shortest_path_with_most_doors.
Test Example 2 passed, for shortest_path_with_most_doors.
Test Example 3 passed, for shortest_path_with_most_doors.
Test Example 4 passed, for shortest_path_with_most_doors.
Test Example 5 passed, for shortest_path_with_most_doors.
Test Example 6 passed, for shortest_path_with_most_doors.
Test Example 7 passed, for shortest_path_with_most_doors.
Test Example 8 passed, for shortest_path_with_most_doors.
Test Example 9 passed, for shortest_path_with_most_doors.
Test Example 10 passed, for shortest_path_with_most_doors.
Test Example 11 passed, for shortest_path_with_most_doors.
Test Example 12 passed, for shortest_path_with_most_doors.
Success


In [47]:
with open("../input/day20.txt") as f:
    puzzle = f.read().strip()

print(f"Part I: {shortest_path_with_most_doors(puzzle)}")

Part I: 3014


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3014</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Okay, so the facility is <span title="Really, really big. No, bigger than that. Even bigger. Keep going. Move. No, more. Look, we're talking krakens and dreadnoughts for housepets. It was big!"><em>big</em></span>.</p>
<p><em>How many rooms have a shortest path from your current location that pass through at least <code>1000</code> doors?</em></p>
</article>

</main>


In [48]:
def count_rooms_shortest_least_limit_doors(regex: str, limit: int = 1_000) -> int:
    deltas = {"N": (-1, 0), "E": (0, 1), "S": (1, 0), "W": (0, -1)}
    positions = deque([])
    distances = {(0, 0): 0}
    r, c = 0, 0
    r1, c1 = r, c
    i = 0

    while regex[i] != "$":
        if regex[i] in "NESW":
            dr, dc = deltas[regex[i]]
            r, c = r + dr, c + dc
            distances[(r, c)] = min(
                distances.get((r, c), inf),
                distances[(r1, c1)] + 1.0,
            )
        elif regex[i] == "(":
            positions.append((r, c))
        elif regex[i] == "|":
            r, c = positions[-1]
        elif regex[i] == ")":
            r, c = positions.pop()

        r1, c1 = r, c
        i += 1

    return sum(1 for d in distances.values() if d >= 1_000)


print(f"Part II: {count_rooms_shortest_least_limit_doors(puzzle)}")  # 9224 too high

Part II: 8279


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>8279</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
